코랩

여기서는 램 12기가까지 지원하는데 얼마나 썼는지 계속 확인할 수 있어서 확인해보니

이제까지 몇십시간씩 걸린건 역시 램이 딸려서 작동을 멈춰버린 상태여서 그런거였네요

여기서도 20만개 한번에 돌리니까 메모리 에러 나서 5만개씩 쪼개서 4번 돌려야겠어요

메모리 문제때문에 pickle로 내렸다가 올렸다가 반복하면서 했습니다

요약하면 ast.literal_eval => convert token to id => padding 했어요

token id로 바꾸는 과정에서 여기서 제공하는 어휘사전이랑 hannanum에서 제공하는 어휘 사전이랑 달라서 인식못하는 단어 => 정수 인코딩에서 0으로 변환하는 경우가 많이 보이더라고요.

만약 모델링을 했는데 성능이 안좋으면

1. tokenizing을 바꾸기

2. token => id로 바꾸는 방법을 바꾸기

등의 방법을 고려해볼 수 있을 것 같아요.

```
pip install kobert-transformers
```

In [1]:
import pickle
import numpy as np
import pandas as pd
import ast
import time

import csv
import torch
import tensorflow as tf

from kobert_transformers import get_tokenizer
# from kobert_transformers import get_kobert_model

```
# 구글 드라이브 마운트 시키기

import os, sys
from google.colab import drive
drive.mount('/content/mnt')
nb_path = '/content/notebooks'
os.symlink('/content/mnt/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)
```

In [2]:
cd /content/mnt/My Drive/Colab Notebooks

/content/mnt/My Drive/Colab Notebooks


In [3]:
data = pd.read_csv('news_normalized.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              200000 non-null  int64 
 1   title           200000 non-null  object
 2   date            200000 non-null  object
 3   body            200000 non-null  object
 4   summary         200000 non-null  object
 5   link            200000 non-null  object
 6   body_morphs     200000 non-null  object
 7   summary_morphs  200000 non-null  object
 8   site            200000 non-null  object
dtypes: int64(1), object(8)
memory usage: 13.7+ MB


# body_morphs에 ast.literal_eval 시켜주기 => token 파일

body_morphs_list_0.pkl

body_morphs_list_1.pkl

body_morphs_list_2.pkl

body_morphs_list_3.pkl

파일 생성

```
body_morphs_list_0 = []

for element in data.body_morphs[:50000]:
    body_morphs_list_0.append(ast.literal_eval(element))
      
with open('body_morphs_list_0.pkl', 'wb') as file:
  pickle.dump(body_morphs_list_0, file)
```


```
body_morphs_list_1 = []

for element in data.body_morphs[50000:100000]:
    body_morphs_list_1.append(ast.literal_eval(element))

with open('body_morphs_list_1.pkl', 'wb') as file:
  pickle.dump(body_morphs_list_1, file)
```


```
body_morphs_list_2 = []

for element in data.body_morphs[100000:150000]:
    body_morphs_list_2.append(ast.literal_eval(element))

with open('body_morphs_list_2.pkl', 'wb') as file:
  pickle.dump(body_morphs_list_2, file)
```


```
body_morphs_list_3 = []

for element in data.body_morphs[150000:]:
    body_morphs_list_3.append(ast.literal_eval(element))

with open('body_morphs_list_3.pkl', 'wb') as file:
  pickle.dump(body_morphs_list_3, file)
```


# 다시 읽어들여서 token을 token id로 변환

bert의 maximum sequence length가 512이므로 token id를 512번째까지만 저장

512 미만으로 token length를 가지고 있는 데이터는 뒤에서 zero padding을 해줄 예정

Hannanum으로 tokenizing을 해서 kobert_tokenizer가 인식 못하는 token은 token_id가 0으로 변환되었음

```
tokenizer = get_tokenizer()
```

```
with open('body_morphs_list_0.pkl', 'rb') as file:
    body_morphs_list_0 = pickle.load(file)

body_morphs_token_id_0 = []

for element in body_morphs_list_0:
  element_id = tokenizer.convert_tokens_to_ids(element)
  body_morphs_token_id_0.append(element_id[:512])

with open('body_morphs_list_1.pkl', 'rb') as file:
    body_morphs_list_1 = pickle.load(file)

body_morphs_token_id_1 = []

for element in body_morphs_list_1:
  element_id = tokenizer.convert_tokens_to_ids(element)
  body_morphs_token_id_1.append(element_id[:512])
```


```
body_morphs_token_id_01 = body_morphs_token_id_0 + body_morphs_token_id_1

with open('body_morphs_token_id_01.pkl', 'wb') as file:
  pickle.dump(body_morphs_token_id_01, file)
```


```
with open('body_morphs_list_2.pkl', 'rb') as file:
    body_morphs_list_2 = pickle.load(file)

body_morphs_token_id_2 = []

for element in body_morphs_list_2:
  element_id = tokenizer.convert_tokens_to_ids(element)
  body_morphs_token_id_2.append(element_id[:512])

with open('body_morphs_list_3.pkl', 'rb') as file:
    body_morphs_list_3 = pickle.load(file)

body_morphs_token_id_3 = []

for element in body_morphs_list_3:
  element_id = tokenizer.convert_tokens_to_ids(element)
  body_morphs_token_id_3.append(element_id[:512])
```


```
body_morphs_token_id_23 = body_morphs_token_id_2 + body_morphs_token_id_3

with open('body_morphs_token_id_23.pkl', 'wb') as file:
  pickle.dump(body_morphs_token_id_23, file)
```


# token_id 불러와서 kobert_embedding 작업

```
with open('body_morphs_token_id_01.pkl', 'rb') as file:
  body_morphs_token_id_01 = pickle.load(file)

with open('body_morphs_token_id_23.pkl', 'rb') as file:
  body_morphs_token_id_23 = pickle.load(file)

body_token_id = body_morphs_token_id_01 + body_morphs_token_id_23
```


```
tokenizer = get_tokenizer()

pad = tokenizer.pad_token_id

print(pad) # 출력값 : 1

# dir(tokenizer)
```


# zero padding

```
# 길이가 512 미만인 애들 뒤쪽에 padding
# 0은 tokenizer의 convert_token_to_id가 모르는 단어, 1은 패딩

body_token_id_padding = []

for element in body_token_id:
  if len(element) < 512:
    pad_element = element + [pad] * (512 - len(element))
  else:
    pad_element = element

  body_token_id_padding.append(pad_element)

with open('body_token_id_padding.pkl', 'wb') as file:
  pickle.dump(body_token_id_padding, file)
```


```
model = get_kobert_model()
```


# embedding 단계 => 모델링에서 하는걸로

```
body_sequence_output = tf.convert_to_tensor(model(torch.LongTensor([body_token_id_padding[0]]))[0].detach())
```


```
start = time.time()
for number in range(1, len(body_token_id_padding[0:20000])):
  body_sequence_output = tf.concat([body_sequence_output,
                                    tf.convert_to_tensor(model(torch.LongTensor([body_token_id_padding[number]]))[0].detach())], axis=0)
  
  if number % 2000 == 0:
    now = time.time()
    print('{}번째 작업, 걸린 시간 : {}분 {}초'.format(number,
                                            round((now - start) // 60),
                                            round((now - start) % 60)))
```


# summary도 body와 똑같이 해주기

```
summary_morphs_list = []
 
for element in data.summary_morphs:
    summary_morphs_list.append(ast.literal_eval(element))
```


```
tokenizer = get_tokenizer()
pad = tokenizer.pad_token_id
```

```
summary_token_id = []

for element in summary_morphs_list:
  element_id = tokenizer.convert_tokens_to_ids(element)
  summary_token_id.append(element_id[:512])
```


```
summary_token_id_padding = []
 
for element in summary_token_id:
  if len(element) < 512:
    pad_element = element + [pad] * (512 - len(element))
  else:
    pad_element = element
 
  summary_token_id_padding.append(pad_element)
 
with open('summary_token_id_padding.pkl', 'wb') as file:
  pickle.dump(summary_token_id_padding, file) 
```
